### minie from split file

> start minie server:
```shell
mvn clean compile exec:java
```

> call example:
```shell
curl 'http://localhost:8080/minie/query' -X POST -d 'Obama visited the white house.' | jq .
```
                                                Post                data                   json

In [1]:
import pickle
import pandas as pd
import spacy
from tqdm import tqdm
import requests

In [2]:
# example
url = 'http://localhost:8080/minie/query'
text = 'Obama visited the white house.'
headers = {'Content-Type': 'text/text; charset=utf-8'}
r = requests.post(url,data = text.encode('utf-8'), headers = headers)
print(r.json())

{'facts': [{'subject': 'Obama', 'predicate': 'visited', 'object': 'white house'}]}


> MinIE.Mode.SAFE  
> Change mode in file FaceResource.java line 30:
```java
MinIE minie = new MinIE(sentence, FactsResource.parser, MinIE.Mode.SAFE);
```

> Two problem here
> 1. https://github.com/uma-pi1/minie/issues/17
> 2. de\uni_mannheim\minie\annotation\AnnotatedPhrase.java line 299

In [19]:
url = 'http://localhost:8080/minie/query'
nlp = spacy.load('en_core_web_lg')

def prepare_text(text):
    doc=nlp(text)
    result=[]
    for sent in list(doc.sents):
            sent=str(sent)
            sent = sent.rstrip() #remove end space
            
            # post to server
            headers = {'Content-Type': 'text/text; charset=utf-8'}
            r = requests.post(url,data = sent.encode('utf-8'), headers = headers)
            
            for facts in r.json()['facts']:
                result.append({'subject':facts['subject'],'relation': facts['predicate'],'object':facts['object']})
    return result

def process(file_name):
    results = []
    var = pickle.load(open('./cleaned_data_split/'+file_name+'.sav', 'rb'))
    for text in tqdm(var):
        results += prepare_text(text)
    result = pd.DataFrame(results)
    result.to_csv('./result_minie/'+file_name+'r.csv',index=False)

In [ ]:
process('a0') # a1 a2 a3 ...

  0%|                                                                            | 22/16042 [05:38<84:55:36, 19.08s/it]

It takes too long, try to process it in java

In [ ]:
# array to json
file_name = 'a1'
var = pickle.load(open('./cleaned_data_split/'+file_name+'.sav', 'rb'))
results = []
nlp = spacy.load('en_core_web_lg')

def to_sent(text):
    doc=nlp(text)
    result=[]
    for sent in list(doc.sents):
        sent=str(sent)
        sent = sent.rstrip() #remove end space
        result.append(sent)
    return result
    
for text in tqdm(var):
    results += to_sent(text)

 82%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                                   | 13114/16042 [05:37<01:27, 33.52it/s]

In [ ]:
import json
json_file = file_name + '.json'
with open('./data_json/'+json_file,'w') as f:
    json.dump(results, f)